In [55]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import seaborn as sns
from thesispy.definitions import ROOT_DIR

INSTANCES_PATH = Path('/home/joasiee/Documents/instances/synthetic/scans/')

In [56]:
def get_img_data(path: Path):
    image = sitk.ReadImage(str(path.resolve()))
    return sitk.GetArrayFromImage(image)

def plot_voxels(data, y_slice_depth = 15, orientation=(0, -70), cmap_name='gray'):
    ax = plt.figure(figsize=(8,8)).add_subplot(projection='3d')
    sliced_data = np.copy(data)
    sliced_data[:, :y_slice_depth, :] = 0
    
    cmap = cm.get_cmap(cmap_name)
    norm = Normalize(vmin=np.min(sliced_data), vmax=np.max(sliced_data))

    colors = np.empty(sliced_data.shape, dtype=object)
    colors = np.array(list(map(lambda x: cmap(norm(x)), sliced_data)))

    ax.voxels(sliced_data, facecolors=colors, edgecolor='k')
    ax.set_xlim3d(2, 28)
    ax.set_ylim3d(5, 28)
    ax.set_zlim3d(2, 28)
    ax.set_box_aspect((np.ptp(ax.get_xlim()), np.ptp(ax.get_ylim()), np.ptp(ax.get_zlim())))
    plt.locator_params(axis='y', nbins=3)
    ax.view_init(*orientation)

def plot_dvf(data, slice=15):
    X, Y = np.indices(data.shape[:2])
    u = data[:, slice, :, 0]
    v = data[:, slice, :, 2]
    colors = np.linalg.norm(data[:, slice, :, [0, 2]], axis=0) # np swaps smallest dim axis to 0
    
    fig, ax = plt.subplots(figsize =(8, 8))
    ax.quiver(X, Y, u, v, colors, angles='xy', scale_units='xy', cmap="Greys")
    plt.show()

In [66]:
from joblib import Parallel, delayed
import numdifftools as nd

def hessian(dvf_slice, p):
  p = np.array(p, dtype=int)
  try:
    dvf_slice[tuple(p)]
  except IndexError:
    print(f"Point {p} is out of bounds")
    return None

  n = len(p)
  output = np.matrix(np.zeros(n*n))
  output = output.reshape(n,n)
  max_indices = np.array(dvf_slice.shape) - 1
  
  for i in range(n):
    for j in range(n):
      ei = np.zeros(n, dtype=int)
      ei[i] = 1
      ej = np.zeros(n, dtype=int)
      ej[j] = 1
      f1 = dvf_slice[tuple(np.clip(p + ei + ej, 0, max_indices))]
      f2 = dvf_slice[tuple(np.clip(p + ei - ej, 0, max_indices))]
      f3 = dvf_slice[tuple(np.clip(p - ei + ej, 0, max_indices))]
      f4 = dvf_slice[tuple(np.clip(p - ei - ej, 0, max_indices))]
      numdiff = (f1-f2-f3+f4)/4
      output[i,j] = numdiff
  return output

def get_hessians_numdiff(dvf):
    hessians = []
    for dim in range(len(dvf.shape)-1):
        dvf_slice = dvf[..., dim]
        max_indices = np.array(dvf_slice.shape) - 1
        dvf_mapping = lambda x: dvf_slice[tuple(np.clip(x.astype(int), 0, max_indices))]
        hessian = nd.Hessian(dvf_mapping, step=1)
        hessians.append(hessian)
    return hessians

def bending_energy(dvf):
  results = Parallel(n_jobs=16)(delayed(bending_energy_point)(dvf, p) for p in np.ndindex(dvf.shape[:-1]))
  return np.sum(results) / np.prod(dvf.shape[:-1])

def bending_energy_point(dvf, p):
  sum = 0.0
  for dim in range(len(dvf.shape)-1):
    sum += np.square(np.linalg.norm(hessian(dvf[..., dim], p)))
  return sum

def bending_energy_numdiff(dvf):
  sum = 0.0
  hessians = get_hessians_numdiff(dvf)
  for p in np.ndindex(dvf.shape[:-1]):
    for dim in range(len(dvf.shape)-1):
      sum += np.square(np.linalg.norm(hessians[dim](p)))
  return sum / np.prod(dvf.shape[:-1])

In [63]:
data = get_img_data(ROOT_DIR / "output" / "1662734354_synthetic_1_adaptivestochasticgradientdescent_314898" / "out" / "deformationField.mhd")
bending_energy(data)

0.005566545035513931

In [65]:
bending_energy(data)

0.00556654503551399

In [53]:
hessians = get_hessians_numdiff(data)
hessians[0]([0,0,0])

array([[-6.77056232e-05,  5.20151480e-05,  5.20151480e-05],
       [ 5.20151480e-05,  4.62580647e-05,  2.28720892e-05],
       [ 5.20151480e-05,  2.28720892e-05,  4.62580647e-05]])